# Workshop Notebook 4: Automation with ML Workload Orchestrations

Wallaroo provides Data Connections and ML Workload Orchestrations to provide organizations with a method of creating and managing automated tasks that can either be run on demand or a regular schedule.

## Prerequisites

* A Wallaroo instance version 2023.2.1 or greater.

## References

* [Wallaroo SDK Essentials Guide: Model Uploads and Registrations: Python Models](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/wallaroo-sdk-model-upload-python/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)
* [Wallaroo SDK Essentials Guide: ML Workload Orchestration](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-ml-workload-orchestration/)

## Orchestrations, Taks, and Tasks Runs

We've details how Wallaroo Connections work.  Now we'll use Orchestrations, Tasks, and Task Runs.

| Item | Description |
|---|---|
| Orchestration | ML Workload orchestration allows data scientists and ML Engineers to automate and scale production ML workflows in Wallaroo to ensure a tight feedback loop and continuous tuning of models from training to production. Wallaroo platform users (data scientists or ML Engineers) have the ability to deploy, automate and scale recurring batch production ML workloads that can ingest data from predefined data sources to run inferences in Wallaroo, chain pipelines, and send inference results to predefined destinations to analyze model insights and assess business outcomes. |
| Task | An implementation of an Orchestration.  Tasks can be either `Run Once`:  They run once and upon completion, stop. `Run Scheduled`: The task runs whenever a specific `cron` like schedule is reached.  Scheduled tasks will run until the `kill` command is issued. |
| Task Run | The execusion of a task.  For `Run Once` tasks, there will be only one `Run Task`.  A `Run Scheduled` tasks will have multiple tasks, one for every time the schedule parameter is met.  Task Runs have their own log files that can be examined to track progress and results. |

## Preliminaries

In the blocks below we will preload some required libraries.

For convenience, the following `helper functions` are defined to retrieve previously created workspaces, models, and pipelines:

* `get_workspace(name, client)`: This takes in the name and the Wallaroo client being used in this session, and returns the workspace matching `name`.  If no workspaces are found matching the name, raises a `KeyError` and returns `None`.
* `get_model_version(model_name, workspace)`: Retrieves the most recent model version from the model matching the `model_name` within the provided `workspace`.  If no model matches that name, raises a `KeyError` and returns `None`.
* `get_pipeline(pipeline_name, workspace)`: Retrieves the most pipeline from the workspace matching the `pipeline_name` within the provided `workspace`.  If no model matches that name, raises a `KeyError` and returns `None`.

In [40]:
import json
import os
import datetime

import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
import numpy as np

pd.set_option('display.max_colwidth', None)

import time
import pyarrow as pa

In [41]:
## convenience functions from the previous notebooks

# return the workspace called <name> through the Wallaroo client.
def get_workspace(name, client):
    workspace = None
    for ws in client.list_workspaces():
        if ws.name() == name:
            workspace= ws
            return workspace
    # if no workspaces were found
    if workspace==None:
        raise KeyError(f"Workspace {name} was not found.")
    return workspace


# returns the most recent model version in a workspace for the matching `model_name`
def get_model_version(model_name, workspace):
    modellist = workspace.models()
    model_version = [m.versions()[-1] for m in modellist if m.name() == model_name]
    # if no models match, return None
    if len(modellist) <= 0:
        raise KeyError(f"Model {mname} not found in this workspace")
        return None
    return model_version[0]

# get a pipeline by name in the workspace
def get_pipeline(pipeline_name, workspace):
    plist = workspace.pipelines()
    pipeline = [p for p in plist if p.name() == pipeline_name]
    if len(pipeline) <= 0:
        raise KeyError(f"Pipeline {pipeline_name} not found in this workspace")
        return None
    return pipeline[0]


### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [42]:
## blank space to log in 

wl = wallaroo.Client()

wallarooPrefix = "doc-test."
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Set Configurations

Set the workspace, pipeline, and model used from Notebook 1.  The helper functions will make this task easier.

#### Set Configurations References

* [Wallaroo SDK Essentials Guide: Workspace Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-workspace/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)

In [43]:
# retrieve the previous workspace, model, and pipeline version

workspace_name = "workshop-workspace-john-06"

workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)

# set your current workspace to the workspace that you just created
wl.set_current_workspace(workspace)

# examine your current workspace
wl.get_current_workspace()

model_name = 'forecast-control-model'

prime_model_version = wl.get_model(model_name)

pipeline_name = 'rental-forecast'

pipeline = wl.get_pipeline(pipeline_name)

# display the workspace, model, and pipeline

display(wl.get_current_workspace())
display(pipeline)
display(prime_model_version)



{'name': 'workshop-workspace-john-06', 'id': 21, 'archived': False, 'created_by': 'aa707604-ec80-495a-a9a1-87774c8086d5', 'created_at': '2023-09-18T18:44:19.80801+00:00', 'models': [{'name': 'forecast-control-model', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 9, 18, 18, 45, 52, 142938, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 9, 18, 18, 45, 52, 142938, tzinfo=tzutc())}], 'pipelines': [{'name': 'rental-forecast', 'create_time': datetime.datetime(2023, 9, 18, 18, 45, 54, 696754, tzinfo=tzutc()), 'definition': '[]'}]}

name,rental-forecast
created,2023-09-18 18:45:54.696754+00:00
last_updated,2023-09-18 18:48:55.908461+00:00
deployed,False
tags,
versions,"554c5e14-3af0-413e-9109-3e9170cb73b3, 14008dde-a730-4b4d-a871-258de93a0c37, a993e136-d4b2-4e80-a8dd-fa8d0e21c929, 3609a78d-8b58-4ff6-a826-c18b7c044901"
steps,forecast-control-model


Name,forecast-control-model
Version,96098ddf-3db2-467d-afdd-397b4a92cdba
File Name,forecast_standard.py
SHA,3cd2acdd1f513f46615be7aa5beac16f09903be851e91f20f6dcdead4a48faa0
Status,ready
Image Path,None
Updated At,2023-18-Sep 18:45:52


## Deploy the Pipeline with the Model Version Step

As per the other workshops:

1. Clear the pipeline of all steps.
1. Add the model version as a pipeline step.
1. Deploy the pipeline with the following deployment configuration:

```python
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
```

In [47]:
pipeline.clear()
pipeline.add_model_step(prime_model_version)

deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
pipeline.deploy(deployment_config=deploy_config)

name,rental-forecast
created,2023-09-18 18:45:54.696754+00:00
last_updated,2023-09-18 18:52:16.985275+00:00
deployed,True
tags,
versions,"edf387cc-927f-4658-ae7d-934f13e66a07, 9599da02-cfa3-4d7f-9472-ba71ee938911, 9ccb0577-feac-4b47-ac9b-45a6abe4b868, 554c5e14-3af0-413e-9109-3e9170cb73b3, 14008dde-a730-4b4d-a871-258de93a0c37, a993e136-d4b2-4e80-a8dd-fa8d0e21c929, 3609a78d-8b58-4ff6-a826-c18b7c044901"
steps,forecast-control-model


### Sample Inference

Verify the pipeline is deployed properly with a sample inference with the file `./data/test_data.df.json`.

In [48]:
# sample inference from previous code here

single_result = pipeline.infer_from_file('../data/testdata-standard.df.json')
display(single_result)

,time,in.count,out.forecast,out.weekly_average,check_failures
0,2023-09-18 18:52:34.670,"[1526, 1550, 1708, 1005, 1623, 1712, 1530, 1605, 1538, 1746, 1472, 1589, 1913, 1815, 2115, 2475, 2927, 1635, 1812, 1107, 1450, 1917, 1807, 1461, 1969, 2402, 1446, 1851]","[1764, 1749, 1743, 1741, 1740, 1740, 1740]",[1745.2857142857142],0


## Sample Orchestration

The orchestration that will automate this process is `./orchestration/real-estate-orchestration.zip`.  The files used are stored in the directory `/orchestration/real-estate-orchestration`, created with the command:

`zip -r real-estate-orchestration.zip real-estate-orchestration/*`.

This contains the following:

* `requirements.txt`:  The Python requirements file to specify the following libraries used.  For this example, that will be empty since we will be using the 
* `main.py`: The entry file that uses a deployed pipeline and performs an inference request against it visible from its log files.
* `data/`: Inference data sources.

The `main.py` script performs a workspace and pipeline retrieval, then an inference against the inference input file.

```python
import wallaroo
from wallaroo.object import EntityNotFoundError
import pandas as pd

wl = wallaroo.Client()

# get the arguments
arguments = wl.task_args()

if "workspace_name" in arguments:
    workspace_name = arguments['workspace_name']
else:
    workspace_name="forecast-model-workshop"

if "pipeline_name" in arguments:
    pipeline_name = arguments['pipeline_name']
else:
    pipeline_name="bikedaypipe"

def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        print(f"Pipeline not found:{name}")
    return pipeline

print(f"Workspace: {workspace_name}")
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)

wl.set_current_workspace(workspace)
print(workspace)

# the pipeline is assumed to be deployed
print(f"Pipeline: {pipeline_name}")
pipeline = wl.build_pipeline(pipeline_name)
print(pipeline)

print(pipeline.status())

single_result = pipeline.infer_from_file('./data/testdata-standard.df.json')

results = pipeline.infer(single_result)

print(results)

```

A few things to go over here.  You'll notice this is almost the exact procedures we've been following so far:  we get a workspace and pipeline, pull data from a CSV file, and perform an inference off the data.

This script assumes that the pipeline has already been deployed, and also includes this part:

`arguments = wl.task_args()`

This allows us to pass arguments into a Task created from an Orchestration, so we can specify a different workspace, pipeline, or any other arguments we construct.  This allows orchestrations to be very flexible.

Also, notice that it refers to a specific file:

`inference_result = pipeline.infer_file_file('./data/test_data.df.json')`

In the `forecast-orchestration` directory is the `data` directory with our sample CSV file.  Orchestrations can include additional artifacts.  We could have used a Wallaroo Connection instead, and we encourage you to try that if you want.

## Upload Orchestration

Orchestrations are uploaded with the Wallaroo client `upload_orchestration(path)` method with the following parameters.

| Parameter | Type | Description |
| --- | --- | ---|
| **path** | string (Required) | The path to the ZIP file to be uploaded. |

Once uploaded, the deployment will be prepared and any requirements will be downloaded and installed.  A typical orchestration upload looks like this:

```python
my_orchestration = wl.upload_orchestration(path-to-zip-file)
```

### Upload Orchestration Exercise

Try uploading our orchestration from `./forecast-orchestration/forecast-orchestration.zip` - or make your own and upload it.

Once uploaded, you can check the status with the `status()`.  If using the orchestration example above, that would be `my_orchestration.status()`  This is handy to make into a loop to check the status until is shows `ready`.

Here's an example of uploading the Orchestration file, then a loop that will keep checking the status every 5 seconds until it returns `ready`.

```python
orchestration = wl.upload_orchestration(name="my real example", path="./forecast-orchestration/forecast-orchestration.zip")

while orchestration.status() != 'ready':
    print(orchestration.status())
    time.sleep(5)
```


In [63]:
orchestration = wl.upload_orchestration(name="forecast sample orchestration", 
                                        path="../forecast-orchestration/forecast-orchestration.zip")

while orchestration.status() != 'ready':
    print(orchestration.status())
    time.sleep(5)

pending_packaging
packaging
packaging
packaging
packaging
packaging
packaging
packaging
packaging


## List Orchestrations

Orchestrations are listed with the Wallaroo Client `list_orchestrations()` method.  Orchestrations can be retrieved to a variable by allocated their position in the array - for example:  `orchestration = wl.list_orchestrations()[0]` would return the first orchestration on the list.

### List Orchestrations Exercise

List all of the orchestrations in your Wallaroo instance.  For example, if your client is saved to `wl`, here's some code that would work.

```python
wl.list_orchestrations()
```

In [64]:
# list orchestration here

wl.list_orchestrations()

id,name,status,filename,sha,created at,updated at
8def0992-ff5e-4965-a6f7-0a3aca9c3b49,forecast sample orchestration,ready,forecast-orchestration.zip,ee0de4...8ed4f7,2023-18-Sep 18:53:25,2023-18-Sep 18:54:16
875aae1d-ddf7-4f25-a120-6160668495e2,forecast sample orchestration,ready,forecast-orchestration.zip,b010f7...c737a1,2023-18-Sep 18:56:20,2023-18-Sep 18:57:07


## Retrieve Orchestration from List

The command `wallaroo.client.list_orchestrations()` returns a List of orchestrations.  We can assign any of the orchestrations in the list to a variable, then use that for other commands.

## Retrieve Orchestration from List Exercise

Use the `list_orchestrations` command and store the orchestration we just uploaded.

Here's some sample code to get you started that stores the last orchestration in the list to the variable `orchestration_from_list`.

```python
orchestration_from_list = wl.list_orchestrations()[-1]
```

In [65]:
# retrieve the orchestration from the list

orchestration_from_list = wl.list_orchestrations()[-1]
orchestration_from_list

Field,Value
ID,875aae1d-ddf7-4f25-a120-6160668495e2
Name,forecast sample orchestration
File Name,forecast-orchestration.zip
SHA,b010f7c206c0e0ba647e05536c96bb8b0b20b6a74b9be3c60662163032c737a1
Status,ready
Created At,2023-18-Sep 18:56:20
Updated At,2023-18-Sep 18:57:07


## Create Run Once Task from Orchestration

The orchestration is now ready to be implemented as a Wallaroo Task.  We'll just run it once as an example.  This specific Orchestration that creates the Task assumes that the pipeline is deployed, and accepts the arguments:

* workspace_name
* pipeline_name

Tasks are either Run Once, or Run Scheduled.  We create a new task from the Orchestration with either `run_once(task_name, json_args, timeout)` or with `run_scheduled(name, timeout,schedule,json_args)`.  The schedule is based on the Kubernetes cron scheduler.  For example:

```python
schedule={'42 * * * *'}
```

Runs every 42 minutes and contains the answer to life, the universe, and everything.

Creating a scheduled task might be:

```python
task_scheduled = orchestration.run_scheduled(name="schedule example", 
                                             timeout=600, 
                                             schedule=schedule, 
                                             json_args={"workspace_name": workspace_name, 
                                                        "pipeline_name": pipeline_name})
```


### Create Run Once Task from Orchestration Exercise

Using the uploaded orchestration, create a Run Once task using your workspace and pipeline names as the `json_args`.  Here's an example using the variables set above.

```python
task = orchestration.run_once(name="real estate task", 
                              json_args={"workspace_name":workspace_name,
                                         "pipeline_name":pipeline_name}
                              )
```

In [66]:
# create your task here

task = orchestration.run_once(name="forecast jsh task", 
                              json_args={"workspace_name":workspace_name,
                                         "pipeline_name":pipeline_name}
                              )

## Monitor Task Run with Task Status

The Task is the **schedule** to execute the instructions within the orchestration.  The actual execution of the task is the **task run**.  A Run Once task will create one Task Run, while a Run Scheduled task will generate a new Task Run each time the schedule pattern is set.

The status task is viewed with the task `status()` command, where it is either `pending` (no tasks runs are generated yet), or `started` (a task run has been started).

## Monitor Task Run with Task Status Example

We'll monitor the run first with it's status with the `Task.status()` command.

Get the status of the task, and once it is `started` proceed to the next step.  Try doing it as a `while` loop if you feel confident.  Here's some sample code where the task was saved to the variable `task`.

```python
task.status
```

Or as a loop pausing ever 5 seconds until the task status is `started`.

```python
while task.status() != "started":
    display(task.status())
    time.sleep(5)
```


In [67]:
while task.status() != "started":
    display(task.status())
    time.sleep(5)

'pending'

## List Tasks

The Wallaroo client `list_tasks` method returns a list of tasks, and shows the the last task run status.

### List Tasks Exercise

List the tasks in your Wallaroo instance.  For example, if your Wallaroo client is stored as `wl`, this would show your tasks.

```python
wl.list_tasks()
```

In [68]:
# empty space to list tasks

wl.list_tasks()

id,name,last run status,type,active,schedule,created at,updated at
b1ecfbcf-0e0c-4769-b7d4-9cf8a52b7cdf,forecast jsh task,running,Temporary Run,True,-,2023-18-Sep 18:57:14,2023-18-Sep 18:57:20
90da33ed-8cef-4e18-951b-81745b6dd53e,forecast jsh task,failure,Temporary Run,True,-,2023-18-Sep 18:54:18,2023-18-Sep 18:54:29


## Display Task Run Results

The Task Run is the implementation of the task - the actual running of the script and it's results.  Tasks that are Run Once will only have one Task Run, while a Task set to Run Scheduled will have a Task Run for each time the task is executed.  Each Task Run has its own set of logs and results that are monitored through the Task Run `logs()` method.

First, get the Task Run - this is the actual execution of a Task.  The Task is the **scheduled** run of an Orchestration.  The Task Run is the **implementation** of a scheduled Task.  A Run Once Task while generate one Task Run, while a Scheduled Task generated a new Task Run every time the schedule pattern is met until the Task is killed.

We retrieve the task runs with the Task `last_runs()` method, and assign a single Task Run to a variable by selecting it with the list with `last_runs()[index]`.  If you only have one Task Run from a Task, then you can just set the `index` to 0.

### Display Task Run Results Exercise

Retrieve the task run for our generated task, then start checking the logs for our task run.  It may take longer than 30 seconds to launch the task, so be prepared to the command multiple times until is it displayed.  Store the task into a variable for later use.

Here's a code sample where the task was saved to the variable `task`.

```python
task_run = task.last_runs()[0]
task_run
```



In [69]:
task_run = task.last_runs()[0]
task_run

Field,Value
Task,b1ecfbcf-0e0c-4769-b7d4-9cf8a52b7cdf
Pod ID,55e8febb-a9eb-4246-b525-13e569f280e1
Status,success
Created At,2023-18-Sep 18:57:17
Updated At,2023-18-Sep 18:57:17


The Task Run Status is checked with the `_status` method.  This lets you know if there was a failure or if it ran successfully.  If it didn't, you can still get the task run logs to find out why.

In [70]:
task_run._status

'success'

## Retrieve Task Run Logs

The Task Run logs are retrieved with the Wallaroo task runs `log()`, and shows the outputs of the results.  This is why it's useful to have `print` commands in your code to track what it's doing.

### Retrieve Task Run Logs Exercise

Take the task run and display the logs.  It may take a few minutes for the logs to show up, so you may need to refresh the code below a few times.  Here's a quick example of some code.

```python
task_run.logs()
```

In [72]:
task_run.logs()

['2023-09-18T18:57:22.476074246Z stdout F Workspace: workshop-workspace-john-06',
 '2023-09-18T18:57:22.476124547Z stdout F {\'name\': \'workshop-workspace-john-06\', \'id\': 21, \'archived\': False, \'created_by\': \'aa707604-ec80-495a-a9a1-87774c8086d5\', \'created_at\': \'2023-09-18T18:44:19.80801+00:00\', \'models\': [{\'name\': \'forecast-control-model\', \'versions\': 1, \'owner_id\': \'""\', \'last_update_time\': datetime.datetime(2023, 9, 18, 18, 45, 52, 142938, tzinfo=tzutc()), \'created_at\': datetime.datetime(2023, 9, 18, 18, 45, 52, 142938, tzinfo=tzutc())}], \'pipelines\': [{\'name\': \'rental-forecast\', \'create_time\': datetime.datetime(2023, 9, 18, 18, 45, 54, 696754, tzinfo=tzutc()), \'definition\': \'[]\'}]}',
 '2023-09-18T18:57:22.476132847Z stdout F Pipeline: rental-forecast',
 "2023-09-18T18:57:22.476139247Z stdout F {'name': 'rental-forecast', 'create_time': datetime.datetime(2023, 9, 18, 18, 45, 54, 696754, tzinfo=tzutc()), 'definition': '[]'}",
 '2023-09-18T18:57:22.476153247Z stdout F                      time  ... check_failures',
 "2023-09-18T18:57:22.476145247Z stdout F {'status': 'Running', 'details': [], 'engines': [{'ip': '10.244.3.192', 'name': 'engine-5c5b5cbdd5-4rw4j', 'status': 'Running', 'reason': None, 'details': [], 'pipeline_statuses': {'pipelines': [{'id': 'rental-forecast', 'status': 'Running'}]}, 'model_statuses': {'models': [{'name': 'forecast-control-model', 'version': '96098ddf-3db2-467d-afdd-397b4a92cdba', 'sha': '3cd2acdd1f513f46615be7aa5beac16f09903be851e91f20f6dcdead4a48faa0', 'status': 'Running'}]}}], 'engine_lbs': [{'ip': '10.244.4.228', 'name': 'engine-lb-584f54c899-6rmjv', 'status': 'Running', 'reason': None, 'details': []}], 'sidekicks': []}",
 '2023-09-18T18:57:22.476159747Z stdout F 0 2023-09-18 18:57:22.390  ...              0',
 '2023-09-18T18:57:22.476165147Z stdout F ',
 '2023-09-18T18:57:22.476173947Z stdout F [1 rows x 5 columns]']

### Cleaning up.

Now that the workshop is complete, don't forget to undeploy your pipeline to free up the resources.

In [73]:
pipeline.undeploy()

name,rental-forecast
created,2023-09-18 18:45:54.696754+00:00
last_updated,2023-09-18 18:52:16.985275+00:00
deployed,False
tags,
versions,"edf387cc-927f-4658-ae7d-934f13e66a07, 9599da02-cfa3-4d7f-9472-ba71ee938911, 9ccb0577-feac-4b47-ac9b-45a6abe4b868, 554c5e14-3af0-413e-9109-3e9170cb73b3, 14008dde-a730-4b4d-a871-258de93a0c37, a993e136-d4b2-4e80-a8dd-fa8d0e21c929, 3609a78d-8b58-4ff6-a826-c18b7c044901"
steps,forecast-control-model


You have now walked through setting up a basic assay and running it over historical data.

## Congratulations!
In this workshop you have
* Deployed a single step house price prediction pipeline and sent data to it.
* Uploaded an ML Orchestration into Wallaroo.
* Created a Run Once Task from the Orchestration.
* Viewed the Task Run's status generated from the Task.
* Viewed the Task Run's logs.

Great job! 

